<a href="https://colab.research.google.com/github/jeffheaton/app_deep_learning/blob/main/assignments/assignment_yourname_t81_558_class9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/index.html)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 9 Assignment: Detect Multiple Faces**

**Student Name: Your Name**

# Assignment Instructions

I provide you with five images of people in various poses, such as the following:

![Multiple Faces](https://data.heatonresearch.com/images/wustl/data/AdobeStock_319245189-low.jpg)

As you have seen from module 9, you can detect faces in this image, as you can see here:

![Multiple Faces](https://s3.amazonaws.com/data.heatonresearch.com/images/wustl/class/bound-1.jpg)

Your task for this assignment is to extract the coordinates (x,y) and dimensions (height, width). Extract the dimensions/coordinates for a rectangle around the individual face rectangles. This rectangle will overlap the edges of some of the separate face rectangles. The single rectangle would look similar, though you do not need to draw it. Rather, you will return a data frame of the coordinates and dimensions.

![Multiple Faces](https://s3.amazonaws.com/data.heatonresearch.com/images/wustl/class/bound-2.jpg)

Generate your dataframe from the following images.

* https://data.heatonresearch.com/images/wustl/data/AdobeStock_158302589-low.jpg
* https://data.heatonresearch.com/images/wustl/data/AdobeStock_268797955-low.jpg
* https://data.heatonresearch.com/images/wustl/data/AdobeStock_319245189-low.jpg
* https://data.heatonresearch.com/images/wustl/data/AdobeStock_622573012-low.jpg
* https://data.heatonresearch.com/images/wustl/data/AdobeStock_632061559-low.jpg

Your submitted dataframe should look like this. Make sure to round your numbers and convert to integer. You will need to calculate the width and height.

|x|y|width|height|
|-|-|-|-|
|177|215|614|134|
|316|74|472|231|
|231|59|497|264|
|436|160|167|245|
|140|192|760|252|

# Google CoLab Instructions

If you are using Google CoLab, it will be necessary to mount your GDrive so that you can send your notebook during the submit process. Running the following code will map your GDrive to ```/content/drive```.

In [ ]:
try:
  from google.colab import drive, userdata
  drive.mount('/content/drive', force_remount=True)
  COLAB = True
  print("Note: using Google CoLab")
except:
  print("Note: not using Google CoLab")
  COLAB = False

# Assignment Submission Key - Was sent you first week of class.
# If you are in both classes, this is the same key.
if COLAB:
  # For Colab, add to your "Secrets" (key icon at the left)
  key = userdata.get('T81_558_KEY')
else:
  # If not colab, enter your key here, or use an environment variable.
  # (this is only an example key, use yours)
  key = "Gx5en9cEVvaZnjhdaushddhuhhO4PsI32sgldAXj"

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
import torch
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems.

**It is unlikely that should need to modify this function.**

In [ ]:
import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io
from typing import List, Union

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# course - The course that you are in, currently t81-558 or t81-559.
# no - The assignment class number, should be 1 through 10.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.

def submit(
    data: List[Union[pd.DataFrame, PIL.Image.Image]],
    key: str,
    course: str,
    no: int,
    source_file: str = None
) -> None:
    if source_file is None and '__file__' not in globals():
        raise Exception("Must specify a filename when in a Jupyter notebook.")
    if source_file is None:
        source_file = __file__

    suffix = f'_class{no}'
    if suffix not in source_file:
        raise Exception(f"{suffix} must be part of the filename.")

    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb', '.py']:
        raise Exception(f"Source file is {ext}; must be .py or .ipynb")

    with open(source_file, "rb") as file:
        encoded_python = base64.b64encode(file.read()).decode('ascii')

    payload = []
    for item in data:
        if isinstance(item, PIL.Image.Image):
            buffered = io.BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG': base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif isinstance(item, pd.DataFrame):
            payload.append({'CSV': base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
        else:
            raise ValueError(f"Unsupported data type: {type(item)}")

    response = requests.post(
        "https://api.heatonresearch.com/wu/submit",
        headers={'x-api-key': key},
        json={
            'payload': payload,
            'assignment': no,
            'course': course,
            'ext': ext,
            'py': encoded_python
        }
    )

    if response.status_code == 200:
        print(f"Success: {response.text}")
    else:
        print(f"Failure: {response.text}")

# Install Facenet-Pytorch

In [ ]:
!pip install facenet-pytorch

Note: You will likely need to restart your session after running the previous block. Don't worry, rerunning the previous block is much faster the second time.

# Assignment #9 Sample Code

The following code provides a starting point for this assignment.

In [ ]:
import os
import pandas as pd
import torch
from facenet_pytorch import MTCNN
from PIL import Image
import requests
from io import BytesIO
import torch
from PIL import Image, ImageDraw
from facenet_pytorch import MTCNN

# You must identify your source file.  (modify for your local setup)
file="/content/drive/My Drive/Colab Notebooks/assignment_yourname_t81_558_class9.ipynb"  # Google CoLab
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\assignments\\assignment_yourname_class9.ipynb'  # Windows
# file='/Users/jheaton/projects/t81_558_deep_learning/assignments/assignment_yourname_class9.ipynb'  # Mac/Linux

mtcnn = MTCNN(keep_all=True, device=device)

# Load image
URLs = [
  "https://data.heatonresearch.com/images/wustl/data/AdobeStock_158302589-low.jpg",
  "https://data.heatonresearch.com/images/wustl/data/AdobeStock_268797955-low.jpg",
  "https://data.heatonresearch.com/images/wustl/data/AdobeStock_319245189-low.jpg",
  "https://data.heatonresearch.com/images/wustl/data/AdobeStock_622573012-low.jpg",
  "https://data.heatonresearch.com/images/wustl/data/AdobeStock_632061559-low.jpg"]


## ... continue your code...

## Submit assignment
submit(source_file=file,data=[df_submit],key=key,no=9,course="t81-558")
#